# Getting started with prompty

## 0. Install dependent packages

In [1]:
%pip install --force-reinstall typing-extensions==4.8

Note: you may need to restart the kernel to use updated packages.Collecting typing-extensions==4.8
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.13.2
    Uninstalling typing_extensions-4.13.2:
      Successfully uninstalled typing_extensions-4.13.2



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
typing-inspection 0.4.0 requires typing-extensions>=4.12.0, but you have typing-extensions 4.8.0 which is incompatible.
pydantic 2.11.3 requires typing-extensions>=4.12.2, but you have typing-extensions 4.8.0 which is incompatible.
openai 1.73.0 requires typing-extensions<5,>=4.11, but you have typing-extensions 4.8.0 which is incompatible.


In [1]:
#%%capture --no-stderr # magic command in Jupyter notebooks is used to capture the output of a cell,
#%pip install prompflow-core

## 1. Execute a Prompty

Prompty is a file with .prompty extension for developing prompt template. 
The prompty asset is a markdown file with a modified front matter. 
The front matter is in yaml format that contains a number of metadata fields which defines model configuration and expected inputs of the prompty.

In [1]:
with open("prompty/basic.prompty") as fin:
    print(fin.read())

---
name: Basic Prompt
description: A basic prompt that uses the chat API to answer questions
model:
    api: chat
    configuration:
        type: azure_openai
        azure_deployment: gpt-4-0125-Preview
    parameters:
        max_tokens: 128
        temperature: 0.2
inputs:
  question:
    type: string
sample:
  "question": "Who is the most famous person in the world?"
---
system:
You are an AI assistant who helps people find information.
As the assistant, you answer questions briefly, succinctly. 

user:
{{question}}


In [3]:
import os
from dotenv import load_dotenv

load_dotenv("credentials.env")

True

In [6]:
from promptflow.core import Prompty

# load prompty as a flow
f = Prompty.load(source="prompty/basic.prompty")

# execute the flow as function
result = f(question="What is the capital of France?")
result

'The capital of France is Paris.'

You can override configuration with `AzureOpenAIModelConfiguration` and `OpenAIModelConfiguration`.

In [5]:
from promptflow.core import AzureOpenAIModelConfiguration, OpenAIModelConfiguration

# override configuration with AzureOpenAIModelConfiguration
configuration = AzureOpenAIModelConfiguration(
    #azure_endpoint="${env:AZURE_OPENAI_ENDPOINT}",  # Use ${env:<ENV_NAME>} to surround the environment variable name.
    #api_key="${env:AZURE_OPENAI_API_KEY}",
    azure_deployment="gpt-4o",
)

# override configuration with OpenAIModelConfiguration
# configuration = OpenAIModelConfiguration(
#     base_url="${env:OPENAI_BASE_URL}",
#     api_key="${env:OPENAI_API_KEY}",
#     model="gpt-3.5-turbo"
# )

override_model = {"configuration": configuration, "parameters": {"max_tokens": 512}}

# load prompty as a flow
f = Prompty.load(source="prompty/basic.prompty", model=override_model)

# execute the flow as function
result = f(question="What is the capital of France?")
result

'The capital of France is Paris.'

### Visualize trace by using start_trace

In [7]:
from promptflow.tracing import start_trace

# start a trace session, and print a url for user to check trace
start_trace()

Starting prompt flow service...
Start prompt flow service on 127.0.0.1:23333, version: 1.13.0.
You can stop the prompt flow service with the following command:'pf service stop'.



Re-run below cell will collect a trace in trace UI.

In [8]:
# rerun the function, which will be recorded in the trace
question = "What is the capital of Japan?"
ground_truth = "Tokyo"
result = f(question=question)
result

You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=AzureOpenAI_Advanced&uiTraceId=0x940d19a845a3aed6165fb02aa414bd8f


'Tokyo.'

### Eval the result 

Note: the eval flow returns a `json_object`.

In [10]:
# load prompty as a flow
eval_flow = Prompty.load("prompty/eval.prompty")
# execute the flow as function
result = eval_flow(question=question, ground_truth=ground_truth, answer=result)
result

You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=AzureOpenAI_Advanced&uiTraceId=0xd5fd8d891da39648c15201cb9cff5c1c


{'score': '5', 'explanation': 'Tokyo is indeed the capital of Japan.'}

In [12]:
!pf service stop

Prompt flow service stop on 127.0.0.1:23333.
